In [ ]:
import pandas as pd
import os
import numpy as np

import matplotlib.pyplot as plt


In [ ]:


#피하는 날
def remove_avoid_day(dF,day=[19]):
    index=dF.index.to_list()
    remo=[]
    for i in index:
        for date in day:
            date="-{} ".format(date)
            if date in str(i):
                 remo.append(i)
    dF=dF.drop(remo)
    return dF


#변동성 돌파 전략 
def lerry_williams(name, fee, interval=100, rang=200):
    
    try:
        dfr=pd.read_pickle(name)
    except:
        return 
    
    plt.figure(name)
    
    def cal_lerry(a,b,label,flag=False):
        if b==0:
            df=dfr[a:]
        else:
            df=dfr[a:b]

        if flag:
            df=remove_avoid_day(df)
            pass

        looc=[]
        looc2=[]
        for i in range(30,70,3):
            i/=100
            looc2.append(i)
            df['range']= (df['high'] - df['low']) * i
            df['target'] = df['open'] + df['range'].shift(1)

            df['ror'] = np.where(
                (df['high'] > df['target'])
            ,df['close'] / df['target'] - fee,1)

            ror = df['ror'].cumprod()[-2]
            looc.append(ror)


        plt.plot(looc2,looc,label=label)
        plt.title(name)        
        plt.legend()
        plt.grid(visible=True)        
    
    for i in range(interval, rang+1, interval):
        cal_lerry(-i,0,"{}".format(i))


def all_coin_lerry():
    #전체코인
    path = ".\\Coindata"
    file_list = os.listdir(path)
    for i in file_list:
        lerry_williams(path+"\\"+i,fee=0.0012,interval=20, rang=200)


def major_coin_lerry():
    #원하는 조합만
    path = ".\\Coindata"
    # ["ADA","XRP","ETH","EOS","ETC","XLM"]
    #["AXS",,"ETC","MED","PLA"]:
    for i in  ["AXS","BCHA","MED","PLA"]:
        lerry_williams(path+"\\"+"KRW-"+i+".pkl",fee=0.0012,interval=30, rang=200)
        
major_coin_lerry()

In [ ]:
#변동성 돌파 상승장 전략

path = ".\\Coindata"
ran=range(100,200,3)


#변동성 돌파 + 상승장 : 리스크가 적고 안전
def lerry_williams(name, fee, interval=100, rang=200):
    try:
        dfr=pd.read_pickle(name)
    except:
        return 
        
    plt.figure(name)
    print("\n {}".format(name))
        
    def cal_lerry(a,b,label):
        if b==0:
            df=dfr[a:]
        else:
            df=dfr[a:b]

        looc=[]
        looc2=[]
        for i in ran:
            i/=100
            looc2.append(i) 
            df['bull'] = df['open'] > (df['close'].rolling(window=10).mean().shift(1))

            df['range']= (df['high'] - df['low']) * i
            df['target'] = df['open'] + df['range'].shift(1)
            df['ror'] = np.where((
                    (df['high'] > df['target']) & df['bull'])
                ,df['close'] / df['target'] - fee,1)

            ror = df['ror'].cumprod()[-2]
            looc.append(ror)

        plt.plot(looc2,looc,label=label)
        plt.title(name)
        plt.grid(b=1)
        plt.legend() 
       
        # temp= looc.copy()
        # temp.sort()
        # if temp[0]<2:
        #     pass
        # else:       
        #     plt.plot(looc2,looc,label=label)
        #     plt.title(name)
        #     plt.grid(b=1)
        #     plt.legend() 
       
        
    

    for i in range(interval, rang+1, interval):
        cal_lerry(-i,0,"{}".format(i))



def all_coin_lerry(interval, rang):
    #전체코인
    
    file_list = os.listdir(path)
    for i in file_list:
        lerry_williams(path+"\\"+i,fee=0.0012,interval=interval, rang=rang)


def major_coin_lerry(interval=30, rang=200):
    #메이저 코인만
    # ["ADA","XRP","ETH","EOS","ETC","XLM"]
    #["AXS","BCHA","ETC","MED","PLA"]
    for i in ["AXS","BCHA","ETC","MED","PLA"]:
        lerry_williams(path+"\\"+"KRW-"+i+".pkl",fee=0.0012,interval=interval, rang=rang)


all_coin_lerry(interval=50, rang=200)
# major_coin_lerry(interval=20, rang=400)

In [ ]:
#n프로 작전
def get_percentage():
    path = ".\\coindata"
    file_list = os.listdir(path)
    file_list =[file_list[9]]
    df_re=pd.DataFrame()
    for i in file_list:
        df=pd.read_pickle(path+"\\"+i)
        df=df.drop(["open","high","low","close","close_yesterday"],axis=1)
        df["close_yesterday"]=df["rage_close"].shift(1)
        df["yesterday_rage_high"]=df["rage_high"].shift(1)

        # df.iat[0,3]=0
        # df.iat[0,4]=0
        df.loc[:,"close_color"]=0
        df.loc[(df["close_yesterday"]>0),"close_color"]=1
        df_re=pd.concat([df_re,df])

    df_re=df_re.dropna(axis=0)
    df_re=df_re*100
    df_re["close_color"]=df_re["close_color"]*0.01

    return df_re
copy=get_percentage() 

copy

In [ ]:
# anum=0.012

anum=2.4
thresh=3
##################################################대조군
total=len(copy)
print("총갯수",len(copy))


a=np.where(
    ((copy["rage_high"]>anum)))
total_suc=len(a[0])
theta=total_suc/total
print("a%오른것",total_suc)
print("총갯수",theta,"\n")




################################################실험군(음봉상관관계)
a=np.where(
    ((copy["close_color"]==1)))
one=len(a[0])
print("어제 음봉이 아닌것",one)


a=np.where(
    ((copy["close_color"]==1) & (copy["rage_high"]>anum)))
two=len(a[0])
print("어제 음봉이 아니고, a%오른것",two)
print("음봉 상관관계수",two/one-theta,"\n")





##############################################
a=np.where(
    ((copy["close_color"]==0)))
one=len(a[0])
print("어제 양봉이 아닌것",one)


a=np.where(
    ((copy["close_color"]==0) & (copy["rage_high"]>anum)))
two=len(a[0])
print("어제 양봉이 아니고, a%오른것",two)
print("양봉 상관관계수",two/one-theta,"\n")


##############################################실험군(어제 종가 고기 차이 스레쉬홀드 1/3)

a=np.where(
    (((copy["yesterday_rage_high"]/thresh)>copy["close_yesterday"])))
five_one=len(a[0])
print("종가<(고점/3)",five_one)


a=np.where(
    (((copy["yesterday_rage_high"]/thresh)>copy["close_yesterday"]) & (copy["rage_high"]>anum)))
five_two=len(a[0])
print("종가<(고점/3), a%오른것",five_two)
print("종가 상관계수",five_two/five_one-theta,"\n")



a=np.where(
    (((copy["yesterday_rage_high"]/thresh)<=copy["close_yesterday"])))
six_one=len(a[0])
print("종가>=(고점/3)",six_one)

a=np.where(
    (((copy["yesterday_rage_high"]/thresh)<=copy["close_yesterday"]) & (copy["rage_high"]>anum)))
six_two=len(a[0])
print("종가>=(고점/3), a%오른것",six_two)


print("종가 상관계수",six_two/six_one-theta,"\n")




##############################################실험군(종합, 2.4%오르고, 1/3 스레쉬홀드)
print("종합")
a=np.where(
    ((copy["close_color"]==1) &  ((copy["yesterday_rage_high"]/thresh)>copy["close_yesterday"])  ))
three_one=len(a[0])
print("어제 음봉이 아니고, 종가<(고점/thresh)",three_one)

a=np.where(
    ((copy["close_color"]==1) &((copy["yesterday_rage_high"]/thresh)>copy["close_yesterday"]) & (copy["rage_high"]>anum)))
three_two=len(a[0])
print("어제 음봉이 아니고, 종가<(고점/thresh), a%오른것",three_two)
print("종가 상관계수",three_two/three_one-theta,"\n")


a=np.where(
    ((copy["close_color"]==1) &  ((copy["yesterday_rage_high"]/thresh)<=copy["close_yesterday"]) ))
four_one=len(a[0])
print("어제 음봉이 아니고, 종가>=(고점/thresh)",four_one)

a=np.where(
    ((copy["close_color"]==1) & ((copy["yesterday_rage_high"]/thresh)<=copy["close_yesterday"]) & (copy["rage_high"]>anum)))
four_two=len(a[0])
print("어제 음봉이 아니고, 종가>=(고점/thresh), a%오른것",four_two)

print("종가 상관계수",four_two/four_one-theta,"\n")


print("(최대상승률, 스레시홀드): ",anum,thresh)

#어제 음봉이 아니고, 종가가 스레스홀드 이상이면 내일은 확실히 어느 수준까지 오른다를 증명



In [ ]:


print("예상수입")
a=np.where(
    ((copy["close_color"]==1) & ((copy["yesterday_rage_high"]/thresh)<=copy["close_yesterday"]) & ((copy["close_yesterday"]>4))&((copy["yesterday_rage_high"]<40))&(copy["rage_high"]>anum)))
b=len(a[0])
print("개수    ",b)
print("평균수입 ",anum)
print("총수익  ",(anum-anum/10)*b)
come=(anum-anum/10)*b

print("\n예상 비용")
a=np.where(
    ((copy["close_color"]==1) & ((copy["yesterday_rage_high"]/thresh)<=copy["close_yesterday"]) & ((copy["close_yesterday"]>4))&((copy["yesterday_rage_high"]<40))&(copy["rage_high"]<anum)))
b=len(a[0])
mean=copy.iloc[a]["rage_close"].mean()
out=mean*b

print("개수    ",b)
print("평균마이너스 ",mean)
print("총비용  ",out)

resul=come+out
print(resul)

In [ ]:
#어디까지 떨어지는 지 알아보는 분포
a=np.where(
    ((copy["close_color"]==1) & ((copy["yesterday_rage_high"]/thresh)<=copy["close_yesterday"]) & ((copy["close_yesterday"]>4))&((copy["yesterday_rage_high"]<40))&(copy["rage_high"]>anum)))


a=copy.iloc[a]
low=a["rage_low"]
end=a["rage_close"]

# print (low,end)
plt.scatter(low,end,alpha=0.03)
plt.ylim(-20,80)
plt.xlim(-20,0)
# plt.plot(range(0,31,5),range(0,31,5))

In [ ]:
anum=10

def get_percentage():
    path = ".\\coindata_min3"
    file_list = os.listdir(path)
    df_re=pd.DataFrame()
    for i in file_list:
        df=pd.read_pickle(path+"\\"+i)
        df=df.drop(["open","high","low","close","close_yesterday"],axis=1)
        df["close_yesterday"]=df["rage_close"].shift(1)
        df["yesterday_rage_high"]=df["rage_high"].shift(1)

        # df.iat[0,3]=0
        # df.iat[0,4]=0
        df.loc[:,"close_color"]=0
        df.loc[(df["close_yesterday"]>0),"close_color"]=1
        df_re=pd.concat([df_re,df])

    df_re=df_re.dropna(axis=0)
    df_re=df_re*100
    df_re["close_color"]=df_re["close_color"]*0.01

    return df_re
copy=get_percentage() 
temp=np.where(copy["rage_high"]>=anum)


copy_copy=copy.iloc[temp]
copy_copy=remove_avoid_day(copy_copy)
# for i in copy_copy["rage_close"]:
#     print(i)

# print("---------------")
# for i in copy_copy["rage_high"]:
#     print(i)
# for i in copy_copy[["rage_high","rage_close"]]:
#     print(i)
# abc=copy_copy["rage_high"]
# abc=abc.cumprod()


abc=(copy_copy["rage_close"]-anum
)*0.01+1
abc=abc.cumprod()
abc.to_dict()

In [ ]:
def min3_stra(com):
    print("분기 ",com)

    #임계수치를 못넘음
    temp2=np.where((copy["rage_high"]>=com)&(copy["rage_high"]<=com+1.5))
    copy_copy2=copy.iloc[temp2]
    print(len(temp2[0]))
    a=len(temp2[0])

    #넘음
    temp2=np.where((copy["rage_high"]>=com))
    copy_copy2=copy.iloc[temp2]
    print(len(temp2[0]))
    b=len(temp2[0])
    # print(copy_copy2[])
    print("클수록 좋음",(b-a)/b,"\n")


for i in range(2,20,1):
    min3_stra(i)

##가장 나이스 한건 13프로 인걸로